In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from random import randint

#### Создание класса нейросети

In [2]:
class MyClass(nn.Module):
    def __init__(self, input_dim, num_hidden, output_dim):
        super().__init__()
        self.layer1 = nn.Linear(input_dim, num_hidden)
        self.layer2 = nn.Linear(num_hidden, output_dim)

    def forward(self, x):
        """Прямой проход по НС"""
        u1 = self.layer1(x)
        s1 = F.tanh(u1)

        u2 = self.layer2(s1)
        s2 = F.tanh(u2)

        return s2

In [3]:
model = MyClass(3, 2, 1)
model

MyClass(
  (layer1): Linear(in_features=3, out_features=2, bias=True)
  (layer2): Linear(in_features=2, out_features=1, bias=True)
)

#### Параметры нейросети

In [4]:
gen_p = model.parameters()
list(gen_p)  # потому что это генератор

[Parameter containing:
 tensor([[ 0.5544,  0.3232,  0.0880],
         [ 0.5351, -0.4100,  0.4105]], requires_grad=True),
 Parameter containing:
 tensor([-0.2240, -0.1944], requires_grad=True),
 Parameter containing:
 tensor([[-0.0293,  0.2095]], requires_grad=True),
 Parameter containing:
 tensor([-0.4604], requires_grad=True)]

#### Объявление оптимизатора и функции потерь 

In [5]:
optimizer = optim.RMSprop(params=model.parameters(), lr=0.01)
loss_func = torch.nn.MSELoss()

#### Обучение

In [6]:
model.train()  # переводим сеть в режим обучения

MyClass(
  (layer1): Linear(in_features=3, out_features=2, bias=True)
  (layer2): Linear(in_features=2, out_features=1, bias=True)
)

In [7]:
x_train = torch.FloatTensor([(-1, -1, -1), (-1, -1, 1), (-1, 1, -1), (-1, 1, 1),
                            (1, -1, -1), (1, -1, 1), (1, 1, -1), (1, 1, 1)])
y_train = torch.FloatTensor([-1, 1, -1, 1, -1, 1, -1, -1])

In [8]:
for _ in range(1000):
    k = randint(0, len(x_train)-1)
    y = model(x_train[k])
    loss = loss_func(y, y_train[k])
    y = y.squeeze()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

c:\interpreter2\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


#### Оценка

In [9]:
model.eval()  # возвращаем в режим эксплуатации

MyClass(
  (layer1): Linear(in_features=3, out_features=2, bias=True)
  (layer2): Linear(in_features=2, out_features=1, bias=True)
)

In [10]:
for x, d in zip(x_train, y_train):
    # model.requires_frad_(False)  # так тоже можно отключать градиенты
    with torch.no_grad():  # отключает поддержку локальных градиентов
        y = model(x)
        print(f'Выходное значение НС: {y.data} => {1 if y.data >= 0 else -1}')

Выходное значение НС: tensor([-0.9872]) => -1
Выходное значение НС: tensor([0.9978]) => 1
Выходное значение НС: tensor([-0.9995]) => -1
Выходное значение НС: tensor([0.9835]) => 1
Выходное значение НС: tensor([-0.9995]) => -1
Выходное значение НС: tensor([0.9835]) => 1
Выходное значение НС: tensor([-0.9996]) => -1
Выходное значение НС: tensor([-0.9810]) => -1
